
# Attrition Risk Early‑Warning System (People Analytics)
**Stack:** Python (pandas, scikit‑learn/XGBoost), SHAP, Tableau, Excel

**How to use**
1) Put the **IBM HR Attrition** CSV into `projects/data/` (e.g., `WA_Fn-UseC_-HR-Employee-Attrition.csv`).
2) Run all cells. Outputs land in `../outputs/attrition/`.
3) Use the Tableau guide (`../tableau_guide.md`) to publish the dashboard.
4) Privacy: we **hash IDs** and **suppress small cells (<5)** in aggregates.


In [11]:
%pip -q install xgboost shap pandas scikit-learn matplotlib openpyxl

In [12]:
import os, pandas as pd
p_rel = "projects/data/WA_Fn-UseC_-HR-Employee-Attrition.csv"
print("Exists?", os.path.exists(p_rel))
df = pd.read_csv(p_rel)
print("Loaded shape:", df.shape)
df.head(2)


Exists? True
Loaded shape: (1470, 35)


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7


In [13]:

# Imports & paths
import hashlib, numpy as np, pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import shap, matplotlib.pyplot as plt

DATA_PATH = Path('./projects/data')
OUT_PATH = Path('../outputs/attrition')
OUT_PATH.mkdir(parents=True, exist_ok=True)
SEED = 42


In [14]:
df = pd.read_csv("projects/data/WA_Fn-UseC_-HR-Employee-Attrition.csv")


In [15]:
# privacy id (keep ONLY for export, NOT for modeling)
if 'EmployeeNumber' in df.columns:
    df['employee_hash'] = df['EmployeeNumber'].astype(str).apply(lambda x: hashlib.sha256(x.encode()).hexdigest()[:12])
else:
    df['employee_hash'] = [hashlib.sha256(f'{i}'.encode()).hexdigest()[:12] for i in range(len(df))]

y = (df['Attrition'].astype(str).str.lower().eq('yes')).astype(int)

# drop non-informative + target + the HASH from features
drop_cols = [c for c in ['EmployeeNumber','Over18','StandardHours','EmployeeCount','Attrition','employee_hash'] if c in df.columns]
X = df.drop(columns=drop_cols)

cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()

In [16]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
pre = ColumnTransformer([('cat', ohe, cat_cols)], remainder='passthrough')
clf = XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=4, subsample=0.9, colsample_bytree=0.9, eval_metric='auc', random_state=42)
pipe = Pipeline([('pre', pre), ('clf', clf)]).fit(X_tr, y_tr)

proba = pipe.predict_proba(X_te)[:,1]
pred  = (proba>=0.5).astype(int)
print("AUC:", round(roc_auc_score(y_te, proba),4))
print(classification_report(y_te, pred))

AUC: 0.7895
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       247
           1       0.79      0.23      0.36        47

    accuracy                           0.87       294
   macro avg       0.83      0.61      0.64       294
weighted avg       0.86      0.87      0.84       294



In [17]:
# SHAP
ohe = pipe.named_steps['pre'].named_transformers_['cat']
feats = list(ohe.get_feature_names_out(cat_cols)) + [c for c in X.columns if c in num_cols]
X_tr_trans = pipe.named_steps['pre'].transform(X_tr)
explainer = shap.TreeExplainer(pipe.named_steps['clf'])
shap_values = explainer.shap_values(X_tr_trans)
shap.summary_plot(shap_values, X_tr_trans, feature_names=feats, show=False)
plt.tight_layout()
plt.savefig('outputs/attrition/shap_summary.png', dpi=150)
plt.close()

# save exports
Path('outputs/attrition').mkdir(parents=True, exist_ok=True)
import numpy as np
pd.DataFrame({'feature':feats, 'mean_abs_shap':np.abs(shap_values).mean(axis=0)})\
  .sort_values('mean_abs_shap', ascending=False)\
  .to_csv('outputs/attrition/shap_feature_importance.csv', index=False)

In [18]:
# Tableau exports (with hash kept ONLY in exports)
df_export = X_te.copy()
df_export['Attrition_True'] = y_te.values
df_export['Attrition_Proba'] = proba
# add back hashed ids aligned by index from original df
df_export['employee_hash'] = df.loc[df_export.index, 'employee_hash'].values

if 'YearsAtCompany' in df_export.columns:
    df_export['TenureBand'] = pd.cut(df_export['YearsAtCompany'], bins=[-1,1,3,5,10,40], labels=['<1','1-3','3-5','5-10','10+'])
else:
    df_export['TenureBand'] = 'NA'

group_cols = [c for c in ['Department','JobRole','Manager','TenureBand'] if c in df_export.columns]
agg = df_export.groupby(group_cols, dropna=False).agg(
    n=('Attrition_True','size'),
    risk=('Attrition_Proba','mean'),
    attrition_rate=('Attrition_True','mean')
).reset_index()
agg['suppressed'] = agg['n'] < 5
agg.loc[agg['suppressed'], ['risk','attrition_rate']] = np.nan

agg.to_csv('outputs/attrition/tableau_attrition_groups.csv', index=False)
df_export[['employee_hash','Attrition_Proba','TenureBand'] + [c for c in group_cols if c in df_export.columns]]\
  .to_csv('outputs/attrition/tableau_attrition_individuals.csv', index=False)

print("Wrote updated exports to outputs/attrition/")
print("Top 5 features:\n", pd.read_csv('outputs/attrition/shap_feature_importance.csv').head())

Wrote updated exports to outputs/attrition/
Top 5 features:
               feature  mean_abs_shap
0         OverTime_No       0.653559
1    StockOptionLevel       0.440436
2       MonthlyIncome       0.437698
3    DistanceFromHome       0.409590
4  NumCompaniesWorked       0.378617


/tmp/ipython-input-3696054338.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agg = df_export.groupby(group_cols, dropna=False).agg(
